## Import Dependencies


In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets, transforms, models
import pandas as pd
import os
from PIL import Image

## Create Dataset for local Directories

In [26]:
class CIFAKE_Dataset(Dataset):
    def __init__(self, root_dir, transform=torchvision.transforms.ToTensor()):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = ['REAL', 'FAKE']
        
    def __len__(self):
        total_len = 0
        for cls in self.classes:
            class_dir = os.path.join(self.root_dir, cls)
            total_len += len(os.listdir(class_dir))
        return total_len
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        
        for cls in self.classes:
            class_dir = os.path.join(self.root_dir, cls)
            class_files = os.listdir(class_dir)
            if idx < len(class_files):
                img_name = os.path.join(class_dir, class_files[idx])
                image = Image.open(img_name)
                label = self.classes.index(cls)
                if self.transform:
                    image = self.transform(image)
                return image, label
            else:
                idx -= len(class_files)

### Load the dataset

In [27]:
# Load the dataset from local directory without preprocessing
training_data = CIFAKE_Dataset(root_dir='data/train')
testing_data = CIFAKE_Dataset(root_dir='data/test')
# select only 30% of the dataset with torch.utils.data.Subset
training_data = torch.utils.data.Subset(training_data, torch.randperm(len(training_data))[:int(0.3*len(training_data))])
testing_data =  torch.utils.data.Subset(testing_data,  torch.randperm(len(testing_data)) [:int(0.3*len(testing_data) )])

image_datasets = {'train': training_data, 'val': testing_data}

In [28]:
# Defind the data loader
train_loader = DataLoader(training_data, batch_size=64, shuffle=True)
test_loader = DataLoader(testing_data, batch_size=64, shuffle=True)
dataloaders = {'train': train_loader, 'val': test_loader}

In [29]:
num_classes = 2
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 256),
    nn.ReLU(),
    nn.Linear(256, num_classes),
    nn.Softmax(dim=1)
)

In [30]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)


In [31]:
num_epochs = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)

cuda:0


In [32]:
for epoch in range(num_epochs):
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()
        
        running_loss = 0.0
        corrects = 0
        
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                    
            running_loss += loss.item() * inputs.size(0)
            corrects += torch.sum(preds == labels.data)
            
        epoch_loss = running_loss / len(image_datasets[phase])
        epoch_acc = corrects.double() / len(image_datasets[phase])
        
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

# Save the trained model
torch.save(model.state_dict(), 'resnet50_finetuned.pth')

train Loss: 0.6129 Acc: 0.7139
val Loss: 0.5611 Acc: 0.7615
train Loss: 0.5447 Acc: 0.7720
val Loss: 0.5283 Acc: 0.7875
train Loss: 0.5255 Acc: 0.7847
val Loss: 0.5186 Acc: 0.7885
train Loss: 0.5189 Acc: 0.7892
val Loss: 0.5102 Acc: 0.7995
train Loss: 0.5121 Acc: 0.7953
val Loss: 0.5051 Acc: 0.8053
train Loss: 0.5095 Acc: 0.7951
val Loss: 0.5023 Acc: 0.8057
train Loss: 0.5065 Acc: 0.7979
val Loss: 0.5007 Acc: 0.8060
train Loss: 0.5035 Acc: 0.8021
val Loss: 0.4975 Acc: 0.8063
train Loss: 0.4987 Acc: 0.8068
val Loss: 0.4940 Acc: 0.8132
train Loss: 0.4990 Acc: 0.8054
val Loss: 0.4947 Acc: 0.8102
